In [21]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import torchvision
from torchvision.utils import save_image
import numpy as np
import math

torch.manual_seed(1234)

In [ ]:
def beta_scheduler(x, t, beta_start = 1e-4, beta_end = 0.02, noise_steps = 1000):


    def prepare_noise_schedule():
        return torch.linspace(beta_start, beta_end, noise_steps)


    beta = prepare_noise_schedule()
    alpha = 1. - beta
    alpha_hat = torch.cumprod(alpha, dim=0)

    sqrt_alpha_hat = torch.tensor(torch.sqrt(alpha_hat[t]),)[:, None, None, None]
    sqrt_one_minus_alpha_hat = torch.tensor(torch.sqrt(1 - alpha_hat[t]),)[:, None, None, None]
    Ɛ = torch.randn_like(x)


    return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ



In [19]:
def beta_scheduler(x, t, type='linear'):
    '''
    params:
    x: is the original image in the form of vector.
    beta_start: is the value of init beta value.
    beta_end: is the value of end beta value.
    noise_steps: is the number of time steps.
    t: is the value of time step.
    '''
    '''
    return:
    x_t: is the image at time step t after adding noise Ɛ.
    noise: Ɛ is a mount of noise added into the image.
    '''


    if type == 'linear':
        beta = linear_beta_schedule()
    elif type == 'cosine':
        beta = cosine_beta_schedule()
    else:
        beta = sigmoid_beta_schedule()

    alpha = 1. - beta
    alpha_hat = torch.cumprod(alpha, dim=0)


    sqrt_alpha_hat = torch.sqrt(alpha_hat[t])[:, None, None, None]
    sqrt_one_minus_alpha_hat = torch.sqrt(1 - alpha_hat[t])[:, None, None, None]
    Ɛ = torch.randn_like(x)


    return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * Ɛ, Ɛ




def linear_beta_schedule(beta_start = 1e-4, beta_end = 0.02, noise_steps = 1000):
    return torch.linspace(beta_start, beta_end, noise_steps)




def cosine_beta_schedule(noise_steps = 1000, s = 0.008):
    '''
    params:
    noise_steps: is the number of time steps.
    s: is the value of time step.
    '''
    '''
    return:
    beta_value: the beta value is worked the same as in ddpm
    '''
    steps = noise_steps + 1
    t = torch.linspace(0, noise_steps, steps, dtype = torch.float64) / noise_steps
    alphas_cumprod = torch.cos((t + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])


    return torch.clip(betas, 0, 0.999)




def sigmoid_beta_schedule(noise_steps = 1000, start = -3, end = 3, tau = 1, clamp_min = 1e-5):
    '''
    params:
    noise_steps: is the number of time steps.
    start: is the begin value of beta scheduler.
    end: is the final value of beta scheduler.
    tau: is the param of sigmoid function.
    clamp_min: is the smallest value which beta is gotten.
    '''
    '''
    return:
    beta_value: the beta value is worked the same as in ddpm
    '''
    steps = noise_steps + 1
    t = torch.linspace(0, noise_steps, steps, dtype = torch.float64) / noise_steps
    v_start = torch.tensor(start / tau).sigmoid()
    v_end = torch.tensor(end / tau).sigmoid()
    alphas_cumprod = (-((t * (end - start) + start) / tau).sigmoid() + v_end) / (v_end - v_start)
    alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])


    return torch.clip(betas, 0, 0.999)

In [23]:
image = Image.open("dog6.jpg")

transforms = torchvision.transforms.Compose([
                torchvision.transforms.Resize(80),  # args.image_size + 1/4 *args.image_size
                torchvision.transforms.RandomResizedCrop(64, scale=(0.8, 1.0)),
                torchvision.transforms.ToTensor(),
                torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                ])

image = transforms(image)
t = torch.Tensor([50, 100, 150, 200, 300, 600, 700, 999]).long()

t = torch.linspace(50, 999, 24).long()


noised_image, _ = beta_scheduler(image, t, 'linear')
save_image(noised_image.add(1).mul(0.5), "noise_linear.jpg")

noised_image, _ = beta_scheduler(image, t, 'cosine')
save_image(noised_image.add(1).mul(0.5), "noise_cosine.jpg")

noised_image, _ = beta_scheduler(image, t, 'sigmoid')
save_image(noised_image.add(1).mul(0.5), "noise_sigmoid.jpg")